In [229]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import sqlalchemy
del city_population_df
from keys import MySQL_Pass


urls = ["https://en.wikipedia.org/wiki/Berlin", 
        "https://en.wikipedia.org/wiki/Hamburg", 
        "https://en.wikipedia.org/wiki/Munich"]
#url = 'https://en.wikipedia.org/wiki/Berlin'
#url = 'https://en.wikipedia.org/wiki/Hamburg'
#url = 'https://en.wikipedia.org/wiki/Munich'

city_population_df = pd.DataFrame(columns=['City', 'Population', 'Latitude' ,'Longitude', 'Country'])
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    lat = soup.find('span', class_='latitude').get_text()
    longt = soup.find('span', class_='longitude').get_text()
    
    country_row0 = soup.find(lambda tag: tag.name == 'th' and 'Country' in tag.text)
    country = country_row0.find_next().get_text()
    
    population_row1 = soup.find(lambda tag: tag.name == 'th' and 'Population' in tag.text)
    population_row2 = row1.find_next('tr')
    population = population_row2.find_next('td').get_text()
    
    city = soup.find('div', class_='fn org').get_text()
    
    city_df = pd.DataFrame({'City': [city], 
                            'Population': [population], 
                            'Latitude': [lat],
                            'Longitude': [longt],
                            'Country': [country]})
    city_population_df = pd.concat([city_population_df, city_df], ignore_index=True)
    
#city_population_df['Population'] = city_population_df['Population'].str.replace(',', '').astype(float)
#city_population_df
city_population_df

,City,Population,Latitude,Longitude,Country
0,Berlin,"1,512,491",52°31′12″N,13°24′18″E,Germany
1,Hamburg,"1,512,491",53°33′N,10°00′E,Germany
2,Munich,"1,512,491",48°08′15″N,11°34′30″E,Germany


In [215]:
city_population_df

,City,Population,Latitude,Longitude,Country
0,Berlin,"1,512,491",52°31′12″N,13°24′18″E,Germany
1,Hamburg,"1,512,491",53°33′N,10°00′E,Germany
2,Munich,"1,512,491",48°08′15″N,11°34′30″E,Germany


In [298]:
soup.find('div', class_='fn org')

<div class="fn org"><span class="wrap">Munich </span></div>

In [276]:
def get_city_info(urls):
   # del city_population_df
    city_population_df = pd.DataFrame(columns=['city', 'population', 'date', 'lat' ,'long', 'country'])
    i=0
    for url in urls:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        lat = soup.find('span', class_='latitude').get_text()
        longt = soup.find('span', class_='longitude').get_text()

        country_row0 = soup.find(lambda tag: tag.name == 'th' and 'Country' in tag.text)
        country = country_row0.find_next().get_text()

        population_row1 = soup.find(lambda tag: tag.name == 'th' and 'Population' in tag.text)
       # <div class="ib-settlement-fn"><span class="nowrap">&nbsp;</span>(2020)<sup id="cite_
        date = population_row1.find_next('div').get_text()
        population_row2 = population_row1.find_next('tr')
        population = int(population_row2.find_next('td').get_text().replace(',', ''))
        city = soup.find('div', class_='fn org').get_text()
        i=i+1
        data_df = pd.DataFrame({'city_id':[int(i)],
                                'city': [city], 
                                'population': [population],
                                'date':[date],
                                'lat': [lat],
                                'long': [longt],
                                'country': [country]})
        city_population_df = pd.concat([city_population_df, data_df], ignore_index=True)
    return city_population_df
city_population_df

,city,population,date,latitude,longitude,country


In [287]:
#city_population_df = pd.DataFrame(columns=['city', 'population', 'date', 'latitude' ,'longitude', 'country'])
city_names = [
    "Berlin", 
    "Hamburg", 
    "Munich"
   #"Stuttgart",
    #"Cairo",
    #"Amman",
    #"São_Paulo",
    #"Mexico_City",
    #"Doha",
   # "Tunis",
   # "Algiers",
   # "Cologne",
   # "Frankfurt",
   # "Leipzig",
   # "Duesseldorf",
   #"Dortmund",
   # "Chicago",
   # "London",
  #  "Tripoli,_Libya"
    #"Paris"
    #"Alexandria"
]

urls = ["https://en.wikipedia.org/wiki/" + city.replace(" ", "_") for city in city_names]

city_data_df = get_city_info(urls)
city_data_df

,city,population,date,lat,long,country,city_id
0,Berlin,3576873,(2024)[4],52°31′12″N,13°24′18″E,Germany,1.0
1,Hamburg,1945532,(2022-12-31)[2],53°33′N,10°00′E,Germany,2.0
2,Munich,1512491,(2022-12-31)[2],48°08′15″N,11°34′30″E,Germany,3.0


In [289]:
info_df = city_data_df[['city_id','population','date','lat','long']].copy()
city_df = city_data_df[['city_id','city']].copy()
city_df['city_id'] = city_df['city_id'].astype(int)
info_df

,city_id,population,date,lat,long
0,1.0,3576873,(2024)[4],52°31′12″N,13°24′18″E
1,2.0,1945532,(2022-12-31)[2],53°33′N,10°00′E
2,3.0,1512491,(2022-12-31)[2],48°08′15″N,11°34′30″E


In [236]:
city_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   city_id  3 non-null      int64 
 1   city     3 non-null      object
dtypes: int64(1), object(1)
memory usage: 180.0+ bytes


In [301]:
schema = "sql_population"
host = "127.0.0.1"
user = "root"
password = MySQL_Pass
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [297]:
city_df['city'][2]

'Munich '

In [309]:
city_df['city'][2] = city_df['city'][2].strip(" ")

/var/folders/b3/vd9cj9tj3kn7lctcbkw7rbw00000gp/T/ipykernel_54021/1788939096.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_df['city'][2] = city_df['city'][2].strip(" ")


In [310]:
city_df['city'][2]

'Munich'

In [311]:
city_df.to_sql('cities',
                  if_exists='append',
                  con=connection_string,
                  index=False)

3

In [304]:
info_df.to_sql('facts',
                  if_exists='append',
                  con=connection_string,
                  index=False)

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'date' in 'field list'")
[SQL: INSERT INTO facts (city_id, population, date, lat, `long`) VALUES (%(city_id)s, %(population)s, %(date)s, %(lat)s, %(long)s)]
[parameters: ({'city_id': 1.0, 'population': 3576873, 'date': '\xa0(2024)[4]', 'lat': '52°31′12″N', 'long': '13°24′18″E'}, {'city_id': 2.0, 'population': 1945532, 'date': '\xa0(2022-12-31)[2]', 'lat': '53°33′N', 'long': '10°00′E'}, {'city_id': 3.0, 'population': 1512491, 'date': '\xa0(2022-12-31)[2]', 'lat': '48°08′15″N', 'long': '11°34′30″E'})]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [286]:
city_df.city[0]

'Berlin'